In [1]:
# Import and create a new SQLContext 
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [2]:
# Read the country CSV file into an RDD.
country_lines = sc.textFile('file:///home/cloudera/Downloads/big-data-3/final-project/country-list.csv')

In [3]:
# Convert each line into a pair of words
country_pairs = country_lines.map(lambda line: line.split(", "))

In [4]:
country_pairs.take(3)

[['Afghanistan', 'AFG'], ['Albania', 'ALB'], ['Algeria', 'ALG']]

In [5]:
# Convert each pair of words into a tuple
country_tuples = country_pairs.map(lambda pair: (pair[0], pair[1]))

In [6]:
country_tuples.take(3)

[('Afghanistan', 'AFG'), ('Albania', 'ALB'), ('Algeria', 'ALG')]

In [7]:
# Create the DataFrame, look at schema and contents
countryDF = sqlContext.createDataFrame(country_tuples, ["country", "code"])
countryDF.printSchema()
countryDF.take(3)

root
 |-- country: string (nullable = true)
 |-- code: string (nullable = true)



[Row(country='Afghanistan', code='AFG'),
 Row(country='Albania', code='ALB'),
 Row(country='Algeria', code='ALG')]

In [8]:
# Read tweets CSV file into RDD of lines
tweet_lines = sc.textFile('file:///home/cloudera/Downloads/big-data-3/final-project/tweet_text_exported.csv')
tweet_lines.count()

13995

In [9]:
# Clean the data: some tweets are empty. Remove the empty tweets using filter() 
# tweet_lines_clean = tweet_lines.filter("tweet_text != ''").select("tweet_text")
tweet_lines_clean = tweet_lines.filter(lambda line: len(line) != 0)
tweet_lines_clean.count()

13391

In [10]:
# Perform WordCount on the cleaned tweet texts. (note: this is several lines.)
wordcount = tweet_lines_clean \
    .flatMap(lambda line: line.split(" ")) \
    .map(lambda word: (word, 1)) \
    .reduceByKey(lambda a, b: (a + b))
wordcount.take(5)

[('', 3292),
 ('https://t.co/fQftAwGAad', 1),
 ('mobile', 1),
 ('#FridayNightTouchdown', 1),
 ('circle', 7)]

In [11]:
# Create the DataFrame of tweet word counts
tweetDF = sqlContext.createDataFrame(wordcount, ["word", "occurences"])
tweetDF.printSchema()

root
 |-- word: string (nullable = true)
 |-- occurences: long (nullable = true)



In [12]:
# Join the country and tweet data frames (on the appropriate column)
merge = tweetDF.join(countryDF, countryDF.country == tweetDF.word)
merge.take(5)

[Row(word='Thailand', occurences=1, country='Thailand', code='THA'),
 Row(word='Iceland', occurences=2, country='Iceland', code='ISL'),
 Row(word='Mexico', occurences=1, country='Mexico', code='MEX'),
 Row(word='Wales', occurences=19, country='Wales', code='WAL'),
 Row(word='Denmark', occurences=1, country='Denmark', code='DEN')]

In [13]:
# Question 1: number of distinct countries mentioned
merge.select('country').distinct().count()

44

In [14]:
# Question 2: number of countries mentioned in tweets.
from pyspark.sql.functions import sum
merge.select(sum('occurences')).show()

+---------------+
|sum(occurences)|
+---------------+
|            397|
+---------------+



In [16]:
# Table 1: top three countries and their counts.
from pyspark.sql.functions import desc
merge.sort(desc("occurences")).show()

+-----------+----------+-----------+----+
|       word|occurences|    country|code|
+-----------+----------+-----------+----+
|     Norway|        52|     Norway| NOR|
|    Nigeria|        49|    Nigeria| NGA|
|     France|        42|     France| FRA|
|   Slovakia|        30|   Slovakia| SVK|
|    England|        25|    England| ENG|
|    Germany|        20|    Germany| GER|
|      Wales|        19|      Wales| WAL|
|     Russia|        15|     Russia| RUS|
|Netherlands|        13|Netherlands| NED|
|     Brazil|        13|     Brazil| BRA|
|     Canada|        11|     Canada| CAN|
|Switzerland|        10|Switzerland| SUI|
|       Chad|         9|       Chad| CHA|
|     Guinea|         8|     Guinea| GUI|
|   Portugal|         8|   Portugal| POR|
|      Spain|         8|      Spain| ESP|
|       Iraq|         6|       Iraq| IRQ|
|     Jordan|         6|     Jordan| JOR|
|    Georgia|         5|    Georgia| GEO|
|      Japan|         5|      Japan| JPN|
+-----------+----------+----------

In [20]:
# Table 2: counts for Kenya, Wales, and Netherlands.
merge.filter(merge.country.isin(['Kenya', 'Wales', 'Netherlands'])).show()

+-----------+----------+-----------+----+
|       word|occurences|    country|code|
+-----------+----------+-----------+----+
|      Wales|        19|      Wales| WAL|
|Netherlands|        13|Netherlands| NED|
|      Kenya|         3|      Kenya| KEN|
+-----------+----------+-----------+----+



In [21]:
from pyspark.sql.functions import mean
merge.select(mean('occurences')).show()

+-----------------+
|  avg(occurences)|
+-----------------+
|9.022727272727273|
+-----------------+

